# Recurrent Neural Networks

This lecture will focus on recurrent neural networks, these are models that take advantage of the sequential nature of some data, like time series, but also sound, and... language!

## What you'll learn in this class 🧐🧐

This course will focus on teaching you the theory and history behind different techniques that will help us analyse sequential data with deep learning.

Here is the outline:

* Introduction
* Recurrent neuron network
  * Simple Recurrent Neuron
  * Long-term dependency problems
* Complex recurrent layers
  * Gated recurrent units
  * LSTMs
* Conclusion

## Introduction

Human beings do not reset their thinking with every passing second, even the most aloof among us! In fact, as you are reading this course, you need to remember the words you just read to understand the complete sentence, let alone what's coming next in the course. You don't instantly erase your memory with each word you read the word you read just before. Your thoughts have a certain lifespan and are organised as logical sequence.

The problem is that classical neural networks do not have this memory capacity, they process each piece of information (image, word, explanatory variable) independently for each observation, regardless of a potential sequential structure. For that reason we need to introduce artificial neurons that can persist some information when analysing sequential data.

For example, the sentence:


<table>
  <tr>
    <td>I</td>
    <td>rose</td>
    <td>from</td>
    <td>the</td>
    <td>ashes</td>
  </tr>
</table>

And this sentence:

<table>
  <tr>
    <td>I</td>
    <td>use</td>
    <td>ashes</td>
    <td>to</td>
    <td>grow</td>
    <td>roses</td>
  </tr>
</table>


Have lots of words in common, by analysing all these words independently it seems unclear how the model would figure out that "rose" is a verb in the first sentence and a noun in the second sentence, this is why taking the context into account is really important.



## Recurrent neural network

Recurrent neural networks are a family of layers that can take into account the sequential nature of input data. Overtime more and more sophisticated sophisticated layers were developped, we'll explore the three main kinds.

### Simple Recurrent Neuron

The simple recurrent neuron is the first type of recurrent that was used in deep learning to analyse sequential data.

The idea is simple, instead of assigning a single weight to all the input's elements and process the full input at once (as would in fully-connected layers) the input will be processed sequentially. The first element from the sequence will go through the layer and produce and output, and this output will be used as an additionnal input when processing the second element from the input sequence and so on.

This principle may be illustrated thanks to the following animation:

![](https://full-stack-assets.s3.eu-west-3.amazonaws.com/images/M08-DeepLearning/NLP/simpleRNN.gif)

As you can see in the animation, the input data (text) is encoded so it becomes a sequence of integers. Then we apply embedding as to convert these arbitrary integers into vector in a given representation space. Note that these two steps are really typical of NLP problems, if we were to analyse time series or sound data we would already be working with sequential numerical data with no need for embedding.

Once the data is presented as a sequence of numerical vectors we can use the simpleRNN layer. One neuron on such a layer has a number of parameters equal to the number of channels in the input data (the input data has shape `(batch_size, sequence_length, channels)`), plus one weight assigned to the hidden state, plus one bias. Mathematically you can see it as:

$$simpleRNN(x^{(l)})=\Phi( \sum_{j=0}^{d}{x_{j}^{(l)} w_j} + h w_{j+1} + b )$$

Where:
* $x^{(l)}$ is the $l^{th}$ vector in the sequence
* $\Phi$ is the activation function of the layer
* $d$ is the number of channels of the input data (in the animation example it's equal to the output dim of the embedding layer)
* $w_0,\dots , w_{j+1}$ are trainable parameters of the layer
* $h$ is the hidden state 
* $b$ is the bias

What happens is that before starting analysing a sequence, the layer is assigning a value to the hidden state $h$, this value can be arbitrary or random. Then after the each element in the sequence goes through the layer, an output value is computed and the hidden state is replace by this output value and fed back to the layer along the next element form the input sequence.

This mechanism lets the layer take into account some hitoric information from the sequence analysis to help compute the outputs for the following elements in the sequence.

Now different options are available in terms of the layer's output, the layer can output the full output sequence (this is very useful if you wish to keep the sequential nature of the data and add another recurrent layer afterwards), but you may also choose to only output the last output which may be interpreted as a summary of the entire information contained in the sequence.

In recent years, recurrent neural networks have demonstrated great performance in a wide range of usecases: speech recognition, language modeling, translation, image captioning, etc... If you want to explore the many possible applications of recurrent neural networks, I encourage you to go to this blog which gives a detailed presentation: [The unreasonnable effectiveness of recurrent neural networks](http://karpathy.github.io/2015/05/21/rnn-effectiveness/).


### Long-term dependency problems

The main advantage of recurrent neural networks is their ability to use previous elements to inform the analysis of following elements of the input, very much like reading the first word of a sentence influences the way you will understand the rest of it. If recurrent networks could do this in a meaningful way, they would be extremely useful, but can they really?

Sometimes it is enough to take into account very recent information to solve the present task. For example, imagine a model that tries to predict the next word from the previous words. If we are trying to predict the last word in the sentence "the clouds are in the _sky_," we don't need more information, it is fairly obvious that the last word in the sentence will be _sky_. In this kind of situation where short term memory is sufficient simpleRNN are very useful.

![](https://full-stack-assets.s3.eu-west-3.amazonaws.com/images/M08-DeepLearning/NLP/ShortTerm.png)

But there are also situations in which a richer or older context is needed in order for the model to perform well. Imagine a problem where you have to guess the last word of the following text: "I grew up in France, near Paris, in a small town called Fontenay-sous-Bois, where I spent eleven years of my life, that's why I am fluent in _French_." In this example, recent information suggests that the last word is definitely a language, but if you want to guess which one it is exactly, you need the context of the beginning of the sentence that talks about France. Typically the longer the sequences, the harder it becomes even for recurrent networks to perform well.

![](https://full-stack-assets.s3.eu-west-3.amazonaws.com/images/M08-DeepLearning/NLP/LongTerm.png)

In theory, recurrent neural networks are capable of managing such "long-term relationships" between elements in the sequence. Unfortunately, in practice, simple recurrent neural networks do not seem to be able to achieve great results. This is why a slightly more complex and much more useful layers have been developped over time.

The long term dependency problem is linked directly to the gradient descent mechanism and more specifically the vanishing gradient problem. Because the of the recurrent properties of the layer, the activation function is applied as many times as there are elements in the sequence, which creates a lot of depth in the network. The priviledged activation function for simpleRNN is `tanh`, because it remains non linear after several composition and does not die (contrary to `ReLu` for example), is symmetrical (treats negative and positive inputs the same way), and is second derivative has nice properties, [find out more here](https://www.quora.com/Why-do-many-recurrent-NNs-use-tanh).

The problem is `tanh` derivative is between zero and one, which means the gradient may only decrease in absolute value after backward propagating through it, combine this with composing the activation functions as many times as the sequence length and you sign yourself up for a vanishing gradient festival! That also means deep recurrent neural networks are really hard to train. Thankfully the layers we will present to you in the second part attempt to overcome that problem.

## Complex recurrent layers

Complex recurrent layer move away from the idea that simply using the previous output to inform future analysis and use more advanced ideas in order to make it easier for the neurons to work with sequential data. (Remember that fully connected neural nets are theoretically capable of all those things, but a lot of the time, adding constraints on the way neurons can train actually improves results because it's actually really hard to train fully connected neural nets without taking advantage of the nature of the data, hence convolutional layers and recurrent layers).

### Gated recurrent units

Gates Recurrent Units or commonly GRU, represent an attempt to solve the vanishing gradient problem that occurs when using simpleRNN.

Simply speaking the GRU solves the vanishing gradient problem by introduction two neuron layers called the **update gate** and **reset gate**. These so-called gates carry trainable weights that let them keep information for a long time or on the contrary filter out information that should not be passed to the output.

We'll introduce some notations for what will follow:

![](https://full-stack-assets.s3.eu-west-3.amazonaws.com/images/M08-DeepLearning/NLP/legend.png)

The GRU layer may be represented in the following way:

* $h_{t-1}$ is the output computed from element $t-1$ in the input sequence
* $x_t$ is the $t^{th}$ element in the sequence
* $r_t$ is the output from the reset gate
* $z_t$ is the update ponderation
* $h^{*}_t$ is the output candidate (it is denoted $\tilde{h}_t$ in the original paper)
* $h_t$ is the output associated to element $t$ of the input sequence

![](https://full-stack-assets.s3.eu-west-3.amazonaws.com/images/M08-DeepLearning/GRU.drawio.png)

We'll take a moment to write the equation and give some interpretation of what is happening in this complex neuron.

#### Reset Gate

In the gated recurrent unit, the output after analysing each element in the sequence also plays the role of "memory" because it is the only element that accompanies the analysis of the full sequence, it is also the hidden state.

The reset gate assigns some weights to the elements in $h_{t-1}$ as well as $x_t$ and applies a sigmoid activation function, so that the output $r_t$ is in the interval $[0,1]$. It's fairly logical that we'd use a sigmoid here since the reset gate's goal is to shrink some information in case we don't want to influence the rest of the operations in the layer.

$$r_t = \sigma(W_r x_t + U_r h_{t-1})$$

* $W_r$ are the parameters associated with the input in the reset gate
* $U_r$ is the parameter associated with the hidden state in the reset gate

#### Update Gate

The update gate is doing two things at once. One part of the update gate is monitoring what proportion of information from the previous hidden state should remain, and what proportion of new information should come into play. The other part is computing a new candidate for the hidden state, which will be used wholy, partly, or not at all depending on ponderation. Let's review the equation together.

$$z_t = \sigma(W_z x_t + U_z h_{t-1})$$

* $W_z$ are the parameters associated with the input in the ponderation part of the update gate.
* $U_z$ is the parameter associated with the hidden state in the poderation part of the update gate.

$$h^*_t = \tanh(W_h x_t + U_h (h_{t-1} \odot r_t))$$

* $W_h$ are the parameters associated with the input sequence in the update gate to build the new hidden state candidate.
* $U_h$ is the parameter associated with the "filtered" previous hidden state.
* $\odot$ means element wise multiplication, not to be confused with vector or matrix multiplication.

The new candidate for the hidden state uses the current element from the input sequence and the previous hidden state that is filtered thanks to the output of the reset gate (which is a sigmoid output between 0 and 1), which means the previous hidden state maybe used entirely, partly, or not at all.

#### Computing the Output and new hidden state

Now let's finally give some details on how the layer will compute its output, the idea is that the output (which is also the hidden state) will be a ponderation between the previous hidden state and the candidate created by the update gate. The ponderation itself is also created by the update gate.

$$h_t = (1 - z_t) h_{t-1} + z_t h^*_t$$

This may seem a little far-fetched, but the fact that instead of one set of weight being responsible for building the output and the hidden state in the simpleRNN, we have three set of trainable parameters that have different roles: One set is taking care of filtering previous information, one is for building the new hidden state candidate, and the last one decides whether the previous hidden state should persist or be replaced by the new cadidate. This is adding a lot of trainable weights in the layer but it makes it easier for the gradient descent algorithm to train them, which explains why these layers achieve better performance working with sequential data.

### LSTMs

LSTM networks are recurrent neural networks that were built to tackle problems invloving long sequences. They were invented by [Hochreiter & Schmidhuber (1997)](http://www.bioinf.jku.at/publications/older/2604.pdf) and have been improved and popularized by many other researchers since then.

#### Central idea of LSTMs

The key to understanding LSTMs is the presence of a new vector which is the cell state, in simpleRNN and GRU the output and the hidden state are the same. In LSTM we use another vector which is the cell state in order to compute the hidden state and output.

This state can be seen as the conveyor belt of an assembly line. This state persists during the entire process with only minor linear transformations, the information generally passes through the cell without being drastically modified.

Similarly to GRUs, LSTMs can be decomposed into gates, except here we have three gates and not two. The idea is the same as for  GRUs, the gates are a way to decompose the way information from current elements and previous elements of the input sequence are used in the layer.

LSTMs may be represented as the following graph:

![](https://full-stack-assets.s3.eu-west-3.amazonaws.com/images/M08-DeepLearning/NLP/LSTM.png)

This may seem a little daunting at first but let's break in down bit by bit together.

#### Forget Gate

The first step in LSTMs is to decide what information will be retained or discarded. This is the role of the **forget gate layer**, it assigns parameters to the previous hidden state and the current element of the input sequence and uses a sigmoid activation function, which outputs values between 0 and 1.

$$f_t = \sigma(W_f x_t + U_f h_{t-1} + b_f)$$

* $f_t$ is the output of the forget gate
* $W_f$ are the parameters associated to the current element in the input sequence
* $U_f$ is the paraeter associated with the previous hidden state
* $x_t$ is element $t$ of the input sequence
* $h_{t-1}$ is the hidden state after analysing element $t-1$
* $b_f$ is the forget gate bias, even though recurrent layers do not necessarily use bias

#### Input Gate

The next step decides what new information will be added to the cell state, and is called the input gate. What the input gate does may be decomposed into two steps, one step creates a filter that will determine what new information may be added to the cell state, and another will compute the new cell state candidate. Let's break this all down by writing the forward pass equations:

$$i_t = \sigma(W_i x_t + U_i h_{t-1} + b_i)$$

This equation represents the computation of the filter that will be applied to the new cell state candidate to limit the information that will go through.

* $i_t$ is the output of the input gate control unit
* $W_i$ are the parameters associated to the input
* $U_i$ is the parameter associated with the previous hidden state
* $b_i$ is the bias

Then it's time to compute the new candidate for the cell state

$$C^*_t = \tanh(W_c x_t + U_c h_{t-1} + b_c)$$

* $C^*_t$ is the candidate new cell state
* $W_c$ are the parameters associated with the input
* $U_c$ is the parameter associated with the previous hidden state
* $b_c$ is the bias 

#### Updating the Cell State

Now that we have studied in detail the forget gate and the input gate, we are ready to understand how the cell state will be updated. What happens is the following:

$$C_t = C_{t-1} \times f_t + C^*_t \times i_t$$

The previous state $C_{t-1}$ gets multiplied by the output $f_t$ of the forget gate, which may result in anything from leaving the previous state unchanged (if $f_t$ is close to 1) to erasing completely (if $f_t$ is close to 0). Then after modifying the previous cell state, we add the new cell state candidate $C^*_t$ multiplied by $i_t$ which is between 0 and 1, meaning we may add all of $C^*_t$ or completely ignore it.

In short, the old information is preserved, or completely, or partially discarded. A new candidate is calculated, but this new information may not be authorized to influence the cell state. Depending on inputs and previous hidden state, the cell state may be preserved or entirely changed at each step when analysing the sequence. Making this capabilities of preserving this "memory", or erasing it and adding or not adding new information explicit tasks of some coponents of the neuron is what is enabling its long-term memory characteristics.

#### Output Gate

Now that the new cell state is calculated, we can determine the ouput of the LSTM layer, it happens in two step, a control unit that will filter the new hidden state candidate, and a hidden state candidate is created from the cell state, let's see how it works in detail.

$$o_t = \sigma(W_o x_t + U_o h_{t-1} + b_o)$$

* $o_t$ is the control unit for calculating the output, it will determine what proportion of the hidden state candidate will go through.
* $W_o$ are parameters associated with the input
* $U_o$ is a paramater associated with the previous hidden state
* $b_o$ is the bias

Then the hidden state (and output of the layer) can be computed:

$$h_t = \tanh(C_t) \times o_t$$

The new hidden state will be fed to the neuron when analysing the next element in the input sequence and will contribute to forming the output vector of the layer.

## Conclusion

Recurrent networks have shown impressive results and helped developping the field of NLP and more broadly sequential data analysis. The main issue with simpleRNN was the risk of vanishing gradient, which made is really difficult for models to train on long sequences. GRU and LSTM layers helped partly lift this difficulty by introducing more complex structure.

However, recurrent neural networks remain very difficult models to train with many pitfalls, thankfully since then, new approaches have been developped which completely changed how people use neural networks on sequential data. This new developement is called **attention layers**, and introduce a fondamental change in the way sequential data is processed by neural networks, instead of processing each element in the sequence one by one, all the elements are taken into consideration at once, but their relative importance is ponderated sequentially according to some context information. We'll have the occasion to study them towards the end of the module.



## Ressources 📚📚

* A great ressource on LSTM : [Understanding LSTM Networks](http://colah.github.io/posts/2015-08-Understanding-LSTMs/).

* [One of the original papers on recurrent neural networks](https://arxiv.org/pdf/1406.1078v3.pdf)

* [An blog post on GRUs](https://towardsdatascience.com/understanding-gru-networks-2ef37df6c9be)

* [A paper on all three kinds of recurrent neurons presented here](https://arxiv.org/pdf/1412.3555v1.pdf)

* [A blog post on LSTM with cool animations](https://towardsdatascience.com/illustrated-guide-to-lstms-and-gru-s-a-step-by-step-explanation-44e9eb85bf21)